## Introduction 

Cette leçon va couvrir : 
- Ce qu'est l'appel de fonction et dans quels cas l'utiliser 
- Comment créer un appel de fonction avec OpenAI 
- Comment intégrer un appel de fonction dans une application 

## Objectifs d'apprentissage 

Après avoir terminé cette leçon, vous saurez comment et comprendrez : 

- L'intérêt d'utiliser l'appel de fonction 
- Configurer un appel de fonction avec le service OpenAI 
- Concevoir des appels de fonction efficaces pour le cas d'usage de votre application


## Comprendre les appels de fonctions

Pour cette leçon, nous voulons créer une fonctionnalité pour notre startup éducative qui permet aux utilisateurs d’utiliser un chatbot pour trouver des cours techniques. Nous recommanderons des cours adaptés à leur niveau de compétence, leur poste actuel et la technologie qui les intéresse.

Pour réaliser cela, nous allons utiliser une combinaison de :
 - `OpenAI` pour créer une expérience de chat pour l’utilisateur
 - `Microsoft Learn Catalog API` pour aider les utilisateurs à trouver des cours selon leur demande
 - `Function Calling` pour prendre la requête de l’utilisateur et l’envoyer à une fonction afin de faire la requête API.

Pour commencer, voyons pourquoi nous voudrions utiliser l’appel de fonction à la base :

print("Messages dans la prochaine requête :")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # obtenir une nouvelle réponse de GPT où il peut voir la réponse de la fonction


print(second_response.choices[0].message)


### Pourquoi l’appel de fonctions

Si vous avez suivi une autre leçon de ce cours, vous comprenez probablement la puissance des grands modèles de langage (LLMs). Vous avez sans doute aussi remarqué certaines de leurs limites.

L’appel de fonctions est une fonctionnalité du service OpenAI conçue pour répondre aux défis suivants :

Formatage de réponse incohérent :
- Avant l’appel de fonctions, les réponses d’un grand modèle de langage étaient non structurées et variables. Les développeurs devaient écrire du code de validation complexe pour gérer chaque variation dans la sortie.

Intégration limitée avec des données externes :
- Avant cette fonctionnalité, il était difficile d’intégrer des données provenant d’autres parties d’une application dans un contexte de chat.

En standardisant les formats de réponse et en permettant une intégration fluide avec des données externes, l’appel de fonctions simplifie le développement et réduit le besoin de logique de validation supplémentaire.

Les utilisateurs ne pouvaient pas obtenir de réponses comme « Quel temps fait-il actuellement à Stockholm ? ». En effet, les modèles étaient limités à la période sur laquelle les données avaient été entraînées.

Regardons l’exemple ci-dessous qui illustre ce problème :

Imaginons que nous souhaitions créer une base de données d’étudiants afin de leur proposer le cours le plus adapté. Ci-dessous, nous avons deux descriptions d’étudiants qui contiennent des informations très similaires.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finshing his studies."

Nous souhaitons envoyer ceci à un LLM pour analyser les données. Cela pourra ensuite être utilisé dans notre application pour l'envoyer à une API ou le stocker dans une base de données.

Créons deux invites identiques dans lesquelles nous indiquons au LLM quelles informations nous intéressent :


Nous voulons envoyer ceci à un LLM pour qu'il analyse les parties importantes pour notre produit. Ainsi, nous pouvons créer deux invites identiques pour instruire le LLM :


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Après avoir créé ces deux invites, nous les enverrons au LLM en utilisant `openai.ChatCompletion`. Nous stockons l'invite dans la variable `messages` et attribuons le rôle à `user`. Cela permet d'imiter un message d'un utilisateur écrit à un chatbot.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Même si les invites sont identiques et que les descriptions se ressemblent, on peut obtenir différents formats pour la propriété `Grades`.

Si vous exécutez la cellule ci-dessus plusieurs fois, le format peut être `3.7` ou `3.7 GPA`.

Cela s’explique par le fait que le LLM prend des données non structurées sous forme d’invite écrite et renvoie également des données non structurées. Il nous faut donc un format structuré afin de savoir à quoi nous attendre lors du stockage ou de l’utilisation de ces données.

En utilisant l’appel de fonctions, on peut s’assurer de recevoir des données structurées en retour. Lorsqu’on utilise l’appel de fonctions, le LLM n’exécute ni n’appelle réellement aucune fonction. À la place, on crée une structure que le LLM doit suivre pour ses réponses. On utilise ensuite ces réponses structurées pour savoir quelle fonction exécuter dans nos applications.


![Diagramme de flux d'appel de fonction](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.fr.png)


### Cas d’utilisation des appels de fonctions

**Appeler des outils externes**  
Les chatbots sont très efficaces pour répondre aux questions des utilisateurs. Grâce à l’appel de fonctions, les chatbots peuvent utiliser les messages des utilisateurs pour accomplir certaines tâches. Par exemple, un étudiant peut demander au chatbot : « Envoie un email à mon professeur pour lui dire que j’ai besoin de plus d’aide sur ce sujet ». Cela peut déclencher un appel à la fonction `send_email(to: string, body: string)`

**Créer des requêtes API ou base de données**  
Les utilisateurs peuvent rechercher des informations en langage naturel, qui sont ensuite transformées en requête formatée ou en appel API. Par exemple, un enseignant peut demander : « Qui sont les étudiants qui ont terminé le dernier devoir ? », ce qui peut appeler une fonction nommée `get_completed(student_name: string, assignment: int, current_status: string)`

**Créer des données structurées**  
Les utilisateurs peuvent prendre un bloc de texte ou un fichier CSV et utiliser le LLM pour en extraire les informations importantes. Par exemple, un étudiant peut transformer un article Wikipédia sur des accords de paix pour créer des fiches de révision avec l’IA. Cela peut se faire en utilisant une fonction appelée `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Créer votre premier appel de fonction

Le processus de création d’un appel de fonction comprend 3 étapes principales :
1. Appeler l’API Chat Completions avec une liste de vos fonctions et un message utilisateur
2. Lire la réponse du modèle pour effectuer une action, c’est-à-dire exécuter une fonction ou un appel d’API
3. Faire un autre appel à l’API Chat Completions avec la réponse de votre fonction afin d’utiliser cette information pour créer une réponse à l’utilisateur.


![Flux d'un appel de fonction](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.fr.png)


### Éléments d’un appel de fonction

#### Saisie de l’utilisateur

La première étape consiste à créer un message utilisateur. Cela peut être attribué dynamiquement en récupérant la valeur d’un champ de saisie texte, ou vous pouvez définir une valeur ici. Si c’est la première fois que vous travaillez avec l’API Chat Completions, il faut définir le `role` et le `content` du message.

Le `role` peut être soit `system` (pour définir des règles), `assistant` (le modèle) ou `user` (l’utilisateur final). Pour l’appel de fonction, nous allons choisir `user` et donner un exemple de question.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Création de fonctions.

Nous allons maintenant définir une fonction ainsi que ses paramètres. Nous allons utiliser ici une seule fonction appelée `search_courses`, mais vous pouvez en créer plusieurs.

**Important** : Les fonctions sont incluses dans le message système envoyé au LLM et seront comptabilisées dans le nombre de tokens disponibles.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Définitions**

La structure de définition de fonction comporte plusieurs niveaux, chacun avec ses propres propriétés. Voici un aperçu de la structure imbriquée :

**Propriétés de la fonction au niveau supérieur :**

`name` - Le nom de la fonction que l’on souhaite appeler.

`description` - Il s’agit de la description du fonctionnement de la fonction. Il est important d’être précis et clair ici.

`parameters` - Une liste des valeurs et du format que vous souhaitez que le modèle produise dans sa réponse.

**Propriétés de l’objet Parameters :**

`type` - Le type de données de l’objet parameters (généralement « object »)

`properties` - Liste des valeurs spécifiques que le modèle utilisera pour sa réponse

**Propriétés de chaque paramètre :**

`name` - Défini implicitement par la clé de propriété (par exemple, « role », « product », « level »)

`type` - Le type de données de ce paramètre spécifique (par exemple, « string », « number », « boolean »)

`description` - Description du paramètre spécifique

**Propriétés optionnelles :**

`required` - Un tableau qui liste les paramètres nécessaires pour que l’appel de la fonction soit complété


### Appeler la fonction
Après avoir défini une fonction, il faut maintenant l’inclure dans l’appel à l’API Chat Completion. Pour cela, on ajoute `functions` à la requête. Dans ce cas, on utilise `functions=functions`.

Il est aussi possible de définir `function_call` sur `auto`. Cela signifie que l’on laisse le LLM choisir quelle fonction appeler en fonction du message de l’utilisateur, au lieu de le faire nous-mêmes.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Voyons maintenant la réponse et examinons son format :

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Vous pouvez voir que le nom de la fonction est appelé et, à partir du message de l'utilisateur, le LLM a pu trouver les données pour remplir les arguments de la fonction.


## 3. Intégrer les appels de fonctions dans une application.

Après avoir testé la réponse formatée du LLM, nous pouvons maintenant l’intégrer dans une application.

### Gérer le flux

Pour intégrer cela dans notre application, suivons les étapes suivantes :

Tout d’abord, faisons l’appel aux services OpenAI et stockons le message dans une variable appelée `response_message`.


In [ ]:
response_message = response.choices[0].message

Nous allons maintenant définir la fonction qui appellera l’API Microsoft Learn pour obtenir une liste de cours :


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



En tant que bonne pratique, nous allons ensuite vérifier si le modèle souhaite appeler une fonction. Ensuite, nous créerons l’une des fonctions disponibles et l’associerons à la fonction appelée.  
Nous prendrons ensuite les arguments de la fonction et les mapperons aux arguments provenant du LLM.

Enfin, nous ajouterons le message d’appel de fonction ainsi que les valeurs retournées par le message `search_courses`. Cela donne au LLM toutes les informations nécessaires pour
répondre à l’utilisateur en langage naturel.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Défi de code

Bravo ! Pour approfondir votre apprentissage sur l’appel de fonctions OpenAI, vous pouvez créer : https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst
 - Plus de paramètres pour la fonction afin d’aider les apprenants à trouver davantage de cours. Vous pouvez consulter les paramètres disponibles de l’API ici :
 - Créez un autre appel de fonction qui prend en compte plus d’informations sur l’apprenant, comme sa langue maternelle
 - Mettez en place une gestion des erreurs lorsque l’appel de fonction et/ou l’appel à l’API ne retourne aucun cours pertinent



---

**Avertissement** :  
Ce document a été traduit à l’aide du service de traduction par IA [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d’assurer l’exactitude de la traduction, veuillez noter que les traductions automatiques peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d’origine doit être considéré comme la source faisant autorité. Pour toute information critique, il est recommandé de recourir à une traduction humaine professionnelle. Nous déclinons toute responsabilité en cas de malentendus ou d’interprétations erronées résultant de l’utilisation de cette traduction.
